# Advanced pyALF part 1: Running ALF

In [1]:
import os
from pprint import pprint  # Pretty print
from urllib.request import urlretrieve  # For downloading the Hamiltonian

from py_alf import ALF_source, Simulation  # Interface with ALF

## ALF_source
Create instance of `ALF_source`, then we download the `Nematic_Dirac` Hamiltonian from the [Hamiltonian repository](https://github.com/ALF-QMC/Hamiltonians/).

Reminder: Directory containing the ALF code is taken from environment variable `$ALF_DIR`, if present.

In [2]:
alf_src = ALF_source()

In [ ]:
ham_name = "Nematic_Dirac"
urlretrieve(
    f'https://raw.githubusercontent.com/ALF-QMC/Hamiltonians/refs/heads/main/Hamiltonians/{ham_name}/Hamiltonian_{ham_name}_smod.F90',
    os.path.join(alf_src.alf_dir, "Prog", "Hamiltonians", f"Hamiltonian_{ham_name}_smod.F90")
)

('/home/jschwab/ALF/Prog/Hamiltonians/Hamiltonian_Nematic_Dirac_smod.F90',
 <http.client.HTTPMessage at 0x70d2c1ce0150>)

In [4]:
with open(os.path.join(alf_src.alf_dir, "Prog", "Hamiltonians.list.d", ham_name), 'w') as f:
    f.write(ham_name + "\n")

Check available Hamiltonians, we have to re-create `alf_src` to register the new Hamiltonian.

In [5]:
alf_src = ALF_source()
alf_src.get_ham_names()

['Kondo',
 'Hubbard',
 'Hubbard_Plain_Vanilla',
 'tV',
 'LRC',
 'Z2_Matter',
 'Spin_Peierls',
 'Nematic_Dirac']

Print valid parameters and their defaults for Nematic Dirac Hamiltonian:

In [6]:
pprint(alf_src.get_default_params('Nematic_Dirac', include_generic=False))

OrderedDict([('VAR_Nematic_Dirac',
              {'Global_J': {'comment': 'J for proposing global updates',
                            'defined_in_base': False,
                            'value': 1.0},
               'Global_h': {'comment': 'h for proposing global updates',
                            'defined_in_base': False,
                            'value': 3.0},
               'Global_type': {'comment': 'Type of global update. Possible '
                                          "values: 'Wolff', 'Geo', 'switch', "
                                          "'flip'",
                               'defined_in_base': False,
                               'value': ''},
               'Ham_J': {'comment': 'Ferromagnetic Ising interaction',
                         'defined_in_base': False,
                         'value': 1.0},
               'Ham_chem': {'comment': 'Chemical potential',
                            'defined_in_base': False,
                            'value': 0

## Parallel Tempering
The simulation object below shows an example of Parallel Tempering, which runs simulations with different parameters in parallel and exchanges configurations between them. The method has been developed to address ergodicity issues, but in this case it's only used to execute a parameter sweep "in one go". For more techincal details on Parallel Tempering, refer to the ALF documentation.

Parallel Tempering is enabled by making `sim_dict` a list of dictionaries, instead of a single dictionary.

In [7]:
for L, time in zip([4, 6, 8, 10], [.2, 1., 7., 18.]):
    print(f'====== L={L} ======')
    sim = Simulation(
        alf_src,
        'Nematic_Dirac',
        [{
            # Model specific parameters
            'Model_vers': 1,
            'L1': L,
            'L2': L,
            'beta': L*4.,
            'Ham_xi': 0.25,
            'Ham_h': h,
            # QMC parameters
            'Ltau': 1,
            'CPU_MAX': time,
            'NSweep': 20,
            'mpi_per_parameter_set': 1,
            'Tempering_calc_det': False,
        } for h in [2.5, 3.0, 3.5, 4.0]],
        machine='gnu',
        mpi=True,
        n_mpi=4,
    )
    if L == 4:
        # We only need to compile once
        sim.compile()
    sim.run()
    sim.print_info_file()

====== L=4 ======
Compiling ALF... 
Cleaning up Prog/
Cleaning up Libraries/
Cleaning up Analysis/
Compiling Libraries


ar: creating modules_90.a


ar: creating libqrref.a


Compiling Analysis


Compiling Program
Parsing Hamiltonian parameters
filenames: Hamiltonians/Hamiltonian_Kondo_smod.F90 Hamiltonians/Hamiltonian_Kondo_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_smod.F90 Hamiltonians/Hamiltonian_Hubbard_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_smod.F90 Hamiltonians/Hamiltonian_Hubbard_Plain_Vanilla_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_tV_smod.F90 Hamiltonians/Hamiltonian_tV_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_LRC_smod.F90 Hamiltonians/Hamiltonian_LRC_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Z2_Matter_smod.F90 Hamiltonians/Hamiltonian_Z2_Matter_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Spin_Peierls_smod.F90 Hamiltonians/Hamiltonian_Spin_Peierls_read_write_parameters.F90
filenames: Hamiltonians/Hamiltonian_Nematic_Dirac_smod.F90 Hamiltonians/Hamiltonian_Nematic_Dirac_read_write_parameters.F90
Compiling program modul

...

===== /home/jschwab/pyALF/doc/source/examples/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=6_L2=6_beta=24.0_xi=0.25_h=2.5/Temp_0/info =====
 Model is            : Nematic_Dirac           1
 Global_type         :                                                                 
 L1                  :            6
 L2                  :            6
 N_SUN               :            2
 ham_t               :    1.0000000000000000     
 dtau                :   0.10000000000000001     
 beta                :    24.000000000000000     
 Ham_h               :    2.5000000000000000     
 Ham_J               :    1.0000000000000000     
 Ham_xi              :   0.25000000000000000     
 Ham_xi2             :    0.0000000000000000     
 Ham_chem            :    0.0000000000000000     
 Global_J            :    1.0000000000000000     
 Global_h            :    3.0000000000000000     
 Phi_1               :    0.0000000000000000     
 Phi_2               :    0.0000000000000000     
 init_type    

 ALF Copyright (C) 2016 - 2022 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration


===== /home/jschwab/pyALF/doc/source/examples/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=8_L2=8_beta=32.0_xi=0.25_h=2.5/Temp_0/info =====
 Model is            : Nematic_Dirac           1
 Global_type         :                                                                 
 L1                  :            8
 L2                  :            8
 N_SUN               :            2
 ham_t               :    1.0000000000000000     
 dtau                :   0.10000000000000001     
 beta                :    32.000000000000000     
 Ham_h               :    2.5000000000000000     
 Ham_J               :    1.0000000000000000     
 Ham_xi              :   0.25000000000000000     
 Ham_xi2             :    0.0000000000000000     
 Ham_chem            :    0.0000000000000000     
 Global_J            :    1.0000000000000000     
 Global_h            :    3.0000000000000000     
 Phi_1               :    0.0000000000000000     
 Phi_2               :    0.0000000000000000     
 init_type    

 ALF Copyright (C) 2016 - 2022 The ALF project contributors
 This Program comes with ABSOLUTELY NO WARRANTY; for details see license.GPL
 This is free software, and you are welcome to redistribute it under certain conditions.
 No initial configuration


===== /home/jschwab/pyALF/doc/source/examples/ALF_data/temper_Nematic_Dirac_Model_vers=1_L1=10_L2=10_beta=40.0_xi=0.25_h=2.5/Temp_0/info =====
 Model is            : Nematic_Dirac           1
 Global_type         :                                                                 
 L1                  :           10
 L2                  :           10
 N_SUN               :            2
 ham_t               :    1.0000000000000000     
 dtau                :   0.10000000000000001     
 beta                :    40.000000000000000     
 Ham_h               :    2.5000000000000000     
 Ham_J               :    1.0000000000000000     
 Ham_xi              :   0.25000000000000000     
 Ham_xi2             :    0.0000000000000000     
 Ham_chem            :    0.0000000000000000     
 Global_J            :    1.0000000000000000     
 Global_h            :    3.0000000000000000     
 Phi_1               :    0.0000000000000000     
 Phi_2               :    0.0000000000000000     
 init_type  

Check info files produced by ALF. It is strongly advised to take a look at these after finished runs, in particular "Precision Green" and "Precision Phase". These should be small, of order $10^{-8}$ or smaller. If they're bigger, one should decrease the stabilization intervall `Nwrap` (see parameter list `'VAR_QMC'` above). In our case, they're of order $10^{-14}$ and one might consider increasing `Nwrap` to speed up the simulation.